In [3]:
import pandas as pd
import numpy as np
from collections import Counter

In [4]:
XLSX_PATH = '../data/Branch_2.xlsx'

In [6]:
sheets = pd.read_excel(XLSX_PATH, sheet_name=[0, 1, 2])

In [213]:
providers, details, supplies = sheets[0], sheets[1], sheets[2]

## Ограничения целостности

### Поставщики 

#### SName NOT NULL
Отбрасывание записи

In [214]:
providers[providers['SName'].isna()].head()

,SID,SName,SCity,Address,Risk
240,240,NaN,Саратов,Учебная 21,3.0
900,900,NaN,﻿Москва,Южная 23,2.0


In [215]:
providers = providers.dropna(subset=['SName'])

In [216]:
providers[providers['SName'].isna()].head()

,SID,SName,SCity,Address,Risk


#### SCity NOT NULL
Замена пустого значения наиболее часто встречающимся значением города поставщика в рамках данного филиала

In [217]:
providers[providers['SCity'].isna()]

,SID,SName,SCity,Address,Risk
540,540,Феникс,NaN,Уральская 37,1.0
660,660,"ООО ""МТА""",NaN,Ереванская 46,2.0
780,780,"ООО ""Эльтадор-М""",NaN,Родькина 47,1.0
1020,1020,СлавКабель,NaN,Литовская 31,1.0


In [218]:
def apply_most_common_constraint(container, column_name, name_column=None):
    container = container.dropna(subset=[column_name])
    def apply_each_row(row):
        row = row.copy()
        if pd.isna(row[column_name]):
            if name_column is not None:
                freq = Counter(container[container[name_column] == row[name_column]][column_name])
            else:
                freq = Counter(container[column_name])
            most_common = freq.most_common()[0][0]
            row[column_name] = most_common
        return row
    return apply_each_row

In [219]:
providers = providers.apply(
    apply_most_common_constraint(providers, 'SCity', 'SName'), 
    axis=1,
)

In [220]:
providers[providers['SCity'].isna()]

,SID,SName,SCity,Address,Risk


#### UNIQUE (SName, Address, SCity)
Отбрасывание записей-дубликатов 

In [221]:
providers[providers.duplicated(subset=['SName', 'Address', 'SCity'], keep=False)].head()

,SID,SName,SCity,Address,Risk
179,179,ВАЛСИ,Новгород,Рылеева 6,2.0
180,180,ВАЛСИ,Новгород,Рылеева 6,3.0
599,599,ВАЛСИ,Ярославль,Южноуральская 32,2.0
600,600,ВАЛСИ,Ярославль,Южноуральская 32,3.0
839,839,Зет-Стоун,Воронеж,Украинская 19,1.0


In [222]:
providers = providers.drop_duplicates(subset=['SName', 'Address', 'SCity'], keep='first')

In [223]:
providers[providers.duplicated(subset=['SName', 'Address', 'SCity'], keep=False)].head()

,SID,SName,SCity,Address,Risk


#### Risk in (1, 2, 3)  
Замена ошибочного значения наиболее часто встречающимся значением риска сотрудничества в рамках данного города данного филиала

In [224]:
possible_risks = [1, 2, 3]

In [225]:
providers[np.logical_not(providers['Risk'].isin(possible_risks))]

,SID,SName,SCity,Address,Risk
60,60,Мотэк-99,Уфа,Житомирская 12,NaN
120,120,Феникс,Ульяновск,Майкова 12,NaN
420,420,﻿Синапсис,Ульяновск,Львовская 22,NaN
480,480,Мотэк-99,Самара,Ростовская 49,NaN
960,960,Феникс,Уфа,Евтеева 14,NaN


In [226]:
providers[np.logical_not(providers['Risk'].isin(possible_risks))].apply(
    apply_most_common_constraint(providers, 'Risk'),
    axis=1,
)

,SID,SName,SCity,Address,Risk
60,60,Мотэк-99,Уфа,Житомирская 12,3.0
120,120,Феникс,Ульяновск,Майкова 12,3.0
420,420,﻿Синапсис,Ульяновск,Львовская 22,3.0
480,480,Мотэк-99,Самара,Ростовская 49,3.0
960,960,Феникс,Уфа,Евтеева 14,3.0


In [227]:
providers[np.logical_not(providers['Risk'].isin(possible_risks))].apply(
    apply_most_common_constraint(providers, 'Risk', 'SName'),
    axis=1,
)

,SID,SName,SCity,Address,Risk
60,60,Мотэк-99,Уфа,Житомирская 12,2.0
120,120,Феникс,Ульяновск,Майкова 12,2.0
420,420,﻿Синапсис,Ульяновск,Львовская 22,2.0
480,480,Мотэк-99,Самара,Ростовская 49,2.0
960,960,Феникс,Уфа,Евтеева 14,2.0


In [228]:
invalid_risk_rows = np.logical_not(providers['Risk'].isin(possible_risks))
providers = providers.apply(
    apply_most_common_constraint(providers, 'Risk', 'SName'),
    axis=1,
)

In [229]:
providers[np.logical_not(providers['Risk'].isin(possible_risks))]

,SID,SName,SCity,Address,Risk


### Детали

In [231]:
details.head()

,PID,PName,PCity,Color,Weight
0,0,﻿Гайка,NaN,Чёрный,0.852
1,1,Болт,Санкт-Петербург,Красный,1.731
2,2,Ремень,Новосибирск,Зелёный,0.866
3,3,Леска,Екатеринбург,Белый,1.166
4,4,Отвёртка,Новгород,Жёлтый,0.162
